In [71]:
def read_file(path):
    res = open(path, "r").read()
    return res.strip()


input = read_file("input")

In [72]:
# input

In [73]:
from math import floor


def expand(disk_map):
    res = ""
    for i, c in enumerate(disk_map):
        if i % 2 != 0:
            res += "." * int(c)
        else:
            file = floor(i / 2)
            res += str(file) * int(c)
    return res


def expand2(disk_map):
    res = []
    for i, c in enumerate(disk_map):
        if i % 2 != 0 and int(c) > 0:
            res.append((-1, int(c)))
        elif int(c) > 0:
            file = floor(i / 2)
            res.append((file, int(c)))

    return res


def disk_map_to_str(disk_map):
    res = ""
    for e in disk_map:
        if e[0] == -1:
            res += "." * e[1]
        else:
            res += str(e[0]) * e[1]
    return res


def is_fragmented(disk_map):
    pairwise = zip(disk_map, disk_map[1:])
    for e in pairwise:
        if e[0][0] == -1 and e[1][0] != -1:
            return True
    return False


def get_first_gap(disk_map):
    for i, c in enumerate(disk_map):
        if c[0] == -1:
            return i


def calc_checksum(disk_map):
    # This needs to calculate the value of each tuple...
    start = 0
    res = 0
    for e in disk_map:
        file_ids = [e[0]] * e[1]
        positions = range(len(file_ids))
        increment = sum(
            [max(0, id) * (pos + start) for id, pos in zip(file_ids, positions)]
        )
        res += increment
        start += len(file_ids)

    return res

In [74]:
disk_map = expand2(input)
while is_fragmented(disk_map):
    tail = disk_map.pop()
    gap_index = get_first_gap(disk_map)
    gap = disk_map[gap_index]
    if gap[1] == tail[1]:
        disk_map[gap_index] = tail
    elif gap[1] > tail[1]:
        reminder = (-1, gap[1] - tail[1])
        disk_map = (
            disk_map[:gap_index] + [tail] + [reminder] + disk_map[gap_index + 1 :]
        )
    else:
        insert = (tail[0], gap[1])
        reminder = (tail[0], tail[1] - gap[1])
        disk_map = (
            disk_map[:gap_index] + [insert] + disk_map[gap_index + 1 :] + [reminder]
        )

    # print(disk_map_to_str(disk_map))

# is_fragmented(expand2(input))

In [75]:
calc_checksum(disk_map)

6200294120911

In [76]:
def get_last_file_id(disk_map):
    return max(disk_map[-1][0], disk_map[-2][0])


def get_file_by_id(disk_map, id):
    for i, e in reversed(list(enumerate(disk_map))):
        if e[0] == id:
            return i


def has_adjecent_free_space(disk_map):
    for t, n in zip(disk_map, disk_map[1:]):
        if t[0] == -1 and n[0] == -1:
            return True


def combine_free_space(disk_map):
    while has_adjecent_free_space(disk_map):
        for i, e in enumerate(disk_map):
            if e[0] == -1 and disk_map[i + 1][0] == -1:
                disk_map[i] = (-1, e[1] + disk_map[i + 1][1])
                disk_map.pop(i + 1)
                break
    return disk_map


disk_map = expand2(input)

max_file_id = get_last_file_id(disk_map)

while max_file_id > 0:
    file_index = get_file_by_id(disk_map, max_file_id)
    file = disk_map[file_index]
    for i, e in enumerate(disk_map):
        if e[0] == -1 and e[1] >= file[1]:
            reminder = (-1, disk_map[i][1] - file[1])
            to_insert = [file]
            if reminder[1] > 0:
                to_insert.append(reminder)
            disk_map = disk_map[:i] + to_insert + disk_map[i + 1 :]
            file_index = get_file_by_id(disk_map, max_file_id)
            disk_map[file_index] = (-1, file[1])
            disk_map = combine_free_space(disk_map)
            break
    max_file_id -= 1

In [77]:
calc_checksum(disk_map)

6227018762750